# SQL Assignment

In [19]:
import pandas as pd
import sqlite3

# from IPython.display import display, HTML

# !gdown --id "1GLoBrN5Iq7yf6lMa4kL0yBMbVTjaRjDV"
# conn = sqlite3.connect("/content/Db-IMDB-Assignment.db")

!gdown --id "1GLoBrN5Iq7yf6lMa4kL0yBMbVTjaRjDV"
conn = sqlite3.connect("/content/Db-IMDB-Assignment.db")


Downloading...
From: https://drive.google.com/uc?id=1GLoBrN5Iq7yf6lMa4kL0yBMbVTjaRjDV
To: /content/Db-IMDB-Assignment.db
7.51MB [00:00, 65.9MB/s]


#### Overview of all tables

In [20]:
tables = pd.read_sql_query("SELECT NAME AS 'Table_Name' FROM sqlite_master WHERE type='table'",conn)
tables = tables["Table_Name"].values.tolist()


In [21]:
for table in tables:
    query = "PRAGMA TABLE_INFO({})".format(table)
    schema = pd.read_sql_query(query,conn)
    print("Schema of",table)
    display(schema)
    print("-"*100)
    print("\n")

Schema of Movie


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,MID,TEXT,0,None,0
2,2,title,TEXT,0,None,0
3,3,year,TEXT,0,None,0
4,4,rating,REAL,0,None,0
5,5,num_votes,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


Schema of Genre


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,Name,TEXT,0,None,0
2,2,GID,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


Schema of Language


...

----------------------------------------------------------------------------------------------------


Schema of M_Director


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,MID,TEXT,0,None,0
2,2,PID,TEXT,0,None,0
3,3,ID,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


Schema of M_Cast


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,MID,TEXT,0,None,0
2,2,PID,TEXT,0,None,0
3,3,ID,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------




## Useful tips:

1. the year column in 'Movie' table, will have few chracters other than numbers which you need to be preprocessed, you need to get a substring of last 4 characters, its better if you convert it as int type, ex: CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER)

2. For almost all the TEXT columns we have show, please try to remove trailing spaces, you need to use TRIM() function

3. When you are doing count(coulmn) it won't consider the "NULL" values, you might need to explore other alternatives like Count(*)

## Q1 --- List all the directors who directed a 'Comedy' movie in a leap year. (You need to check that the genre is 'Comedy’ and year is a leap year) Your query should return director name, the movie name, and the year.

<h4>To determine whether a year is a leap year, follow these steps:</h4>

<ul>
    <li><b>STEP-1:</b> If the year is evenly divisible by 4, go to step 2. Otherwise, go to step 5.</li>
    <li><b>STEP-2:</b> If the year is evenly divisible by 100, go to step 3. Otherwise, go to step 4.</li>
    <li><b>STEP-3:</b> If the year is evenly divisible by 400, go to step 4. Otherwise, go to step 5.</li>
    <li><b>STEP-4:</b> The year is a leap year (it has 366 days).</li>
    <li><b>STEP-5:</b> The year is not a leap year (it has 365 days).</li>
</ul>

Year 1900 is divisible by 4 and 100 but it is not divisible by 400, so it is not a leap year.

In [22]:
%%time
def grader_1(q1):
    q1_results  = pd.read_sql_query(q1,conn)
    print(q1_results.head(10))
    assert (q1_results.shape == (232,3))

query1 = """ 
SELECT 
   m.title, p.name, m.year 
FROM Movie m 
JOIN M_Genre mg ON m.MID = mg.MID
JOIN Genre g ON g.GID = mg.GID
JOIN M_Director d on d.MID = m.MID
JOIN Person p on d.PID = p.PID
WHERE 
  (TRIM(g.Name) LIKE '%COMEDY%') 
    AND
  (
    (CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER) % 400 = 0) OR 
    (CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER) % 4 = 0 AND 
    CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER) % 100 <> 0)
  )
;
"""
grader_1(query1)

                               title               Name  year
0                         Mastizaade       Milap Zaveri  2016
1  Harold & Kumar Go to White Castle       Danny Leiner  2004
2                 Gangs of Wasseypur     Anurag Kashyap  2012
3        Around the World in 80 Days       Frank Coraci  2004
4             The Accidental Husband      Griffin Dunne  2008
5                             Barfi!        Anurag Basu  2012
6                  Bride & Prejudice    Gurinder Chadha  2004
7    Beavis and Butt-Head Do America         Mike Judge  1996
8                            Dostana   Tarun Mansukhani  2008
9                      Kapoor & Sons       Shakun Batra  2016
CPU times: user 66.4 ms, sys: 3.94 ms, total: 70.4 ms
Wall time: 69.8 ms


## Q2 --- List the names of all the actors who played in the movie 'Anand' (1971)

In [23]:
%%time
def grader_2(q2):
    q2_results  = pd.read_sql_query(q2,conn)
    print(q2_results.head(10))
    assert (q2_results.shape == (17,1))

query2 = """
SELECT 
    DISTINCT TRIM(Name) Name 
FROM Person 
WHERE PID IN (
              SELECT 
                  TRIM(PID) 
              FROM M_Cast 
              WHERE MID = (SELECT TRIM(MID) FROM Movie M WHERE TRIM(title) = 'Anand')
            );
 """
grader_2(query2)

               Name
0  Amitabh Bachchan
1     Rajesh Khanna
2     Sumita Sanyal
3        Ramesh Deo
4         Seema Deo
5    Asit Kumar Sen
6        Dev Kishan
7      Atam Prakash
8     Lalita Kumari
9            Savita
CPU times: user 20.8 ms, sys: 995 µs, total: 21.8 ms
Wall time: 21.2 ms


## Q3 --- List all the actors who acted in a film before 1970 and in a film after 1990. (That is: < 1970 and > 1990.)

In [24]:
%%time

def grader_3a(query_less_1970, query_more_1990):
    q3_a = pd.read_sql_query(query_less_1970,conn)
    print(q3_a.shape)
    q3_b = pd.read_sql_query(query_more_1990,conn)
    print(q3_b.shape)
    return (q3_a.shape == (4942,1)) and (q3_b.shape == (62570,1))

query_less_1970 ="""
WITH filterActor AS (
  SELECT 
      trim(mc.PID) as PID 
  FROM M_Cast mc
  WHERE TRIM(mc.MID) IN (
                          SELECT 
                              trim(MID) as MID 
                          FROM Movie
                          WHERE CAST(SUBSTR(TRIM(year),-4) AS INTEGER) < 1970
                        )
)

SELECT
    p.name 
FROM filterActor fa
INNER JOIN Person p ON fa.PID = p.PID;
"""

query_more_1990 ="""
WITH filterActor AS (
  SELECT 
      trim(mc.PID) AS PID 
  FROM M_Cast mc
  WHERE TRIM(mc.MID) IN (
                          SELECT 
                              trim(MID) AS MID 
                          FROM Movie
                          WHERE CAST(SUBSTR(TRIM(year),-4) AS INTEGER) > 1990
                        )
)

SELECT 
    p.name 
FROM filterActor fa
INNER JOIN Person p ON fa.PID = p.PID;
"""
print(grader_3a(query_less_1970, query_more_1990))


(4942, 1)
(62570, 1)
True
CPU times: user 257 ms, sys: 8.93 ms, total: 266 ms
Wall time: 265 ms


In [25]:
%%time
def grader_3(q3):
    q3_results  = pd.read_sql_query(q3,conn)
    print(q3_results.head(10))
    assert (q3_results.shape == (300,1))

query3 = """ 
WITH fab AS (
  SELECT 
      trim(mc.PID) as PID 
  FROM M_Cast mc
  WHERE 
    TRIM(mc.MID) IN (
                      SELECT 
                          trim(MID) AS MID 
                      FROM Movie
                      WHERE CAST(SUBSTR(TRIM(year),-4) AS INTEGER) < 1970
                    )
),
faa AS (
  SELECT 
      trim(mc.PID) as PID 
  FROM M_Cast mc
  WHERE TRIM(mc.MID) IN (
                      SELECT 
                          trim(MID) AS MID 
                      FROM Movie
                      WHERE CAST(SUBSTR(TRIM(year),-4) AS INTEGER) > 1990
                    )
)

SELECT 
    p.name as Actor_Name 
FROM Person p
WHERE TRIM(p.PID) 
IN faa AND TRIM(p.PID) IN fab
;
"""
grader_3(query3)

          Actor_Name
0       Rishi Kapoor
1   Amitabh Bachchan
2             Asrani
3       Zohra Sehgal
4    Parikshat Sahni
5      Rakesh Sharma
6        Sanjay Dutt
7          Ric Young
8              Yusuf
9     Suhasini Mulay
CPU times: user 131 ms, sys: 2.96 ms, total: 134 ms
Wall time: 134 ms


## Q4 --- List all directors who directed 10 movies or more, in descending order of the number of movies they directed. Return the directors' names and the number of movies each of them directed.

In [26]:
%%time

def grader_4a(query_4a):
    query_4a = pd.read_sql_query(query_4a,conn)
    print(query_4a.head(10)) 
    return (query_4a.shape == (1462,2))

query_4a ="""
SELECT 
  PID DIRECTOR_ID, 
  COUNT(MID) MOVIE_COUNT 
FROM M_Director md
GROUP BY TRIM(md.PID);
 """
print(grader_4a(query_4a))

# using the above query, you can write the answer to the given question

  DIRECTOR_ID  MOVIE_COUNT
0   nm0000180            1
1   nm0000187            1
2   nm0000229            1
3   nm0000269            1
4   nm0000386            1
5   nm0000487            2
6   nm0000965            1
7   nm0001060            1
8   nm0001162            1
9   nm0001241            1
True
CPU times: user 11.4 ms, sys: 971 µs, total: 12.3 ms
Wall time: 11.6 ms


In [27]:
%%time
def grader_4(q4):
    q4_results  = pd.read_sql_query(q4,conn)
    print(q4_results.head(10))
    assert (q4_results.shape == (58,2))



query4 = """
SELECT 
    DISTINCT TRIM(P.Name), 
    MC.M_Count 
FROM Person P JOIN (
                      SELECT 
                          TRIM(PID) AS PID,
                          COUNT(MID) AS M_Count 
                      FROM M_Director 
                      GROUP BY PID HAVING COUNT(MID)>=10 ) AS MC 
ON MC.PID = P.PID
ORDER BY MC.M_Count DESC
""" 


grader_4(query4)

           TRIM(P.Name)  M_Count
0          David Dhawan       39
1          Mahesh Bhatt       35
2       Ram Gopal Varma       30
3          Priyadarshan       30
4          Vikram Bhatt       29
5  Hrishikesh Mukherjee       27
6           Yash Chopra       21
7        Shakti Samanta       19
8       Basu Chatterjee       19
9          Subhash Ghai       18
CPU times: user 22.5 ms, sys: 0 ns, total: 22.5 ms
Wall time: 21.7 ms


## Q5.a --- For each year, count the number of movies in that year that had only female actors.

In [28]:
%%time

# note that you don't need TRIM for person table

def grader_5aa(query_5aa):
    query_5aa = pd.read_sql_query(query_5aa,conn)
    print(query_5aa.head(10)) 
    return (query_5aa.shape == (8846,3))

# query_5aa =""" *** Write your query that will get movie id, and number of people for each geneder *** """
query_5aa = """
WITH comb AS (
  SELECT *
  FROM m_cast mc 
  JOIN Person p ON p.PID = trim(mc.PID)
)

SELECT 
    MID, 
    Gender, 
    count(*) as count
FROM comb
GROUP BY 
    trim(MID),
    Gender ;
"""
print(grader_5aa(query_5aa))

def grader_5ab(query_5ab):
    query_5ab = pd.read_sql_query(query_5ab,conn)
    print(query_5ab.head(10)) 
    return (query_5ab.shape == (3469, 3))

# query_5ab =""" *** Write your query that will have at least one male actor try to use query that you have written above *** """
query_5ab = """ 
WITH comb AS (
  SELECT *
  FROM m_cast mc 
  JOIN Person p ON p.PID = trim(mc.PID)
)

SELECT 
    MID, 
    Gender AS gender, 
    count(*) AS act_count 
FROM comb
GROUP BY 
    trim(MID),
    Gender 
HAVING Gender='Male' and act_count >= 0;
"""

print(grader_5ab(query_5ab))


# using the above queries, you can write the answer to the given question

         MID  Gender  count
0  tt0021594    None      1
1  tt0021594  Female      3
2  tt0021594    Male      5
3  tt0026274    None      2
4  tt0026274  Female     11
5  tt0026274    Male      9
6  tt0027256    None      2
7  tt0027256  Female      5
8  tt0027256    Male      8
9  tt0028217  Female      3
True
         MID gender  act_count
0  tt0021594   Male          5
1  tt0026274   Male          9
2  tt0027256   Male          8
3  tt0028217   Male          7
4  tt0031580   Male         27
5  tt0033616   Male         46
6  tt0036077   Male         11
7  tt0038491   Male          7
8  tt0039654   Male          6
9  tt0040067   Male         10
True
CPU times: user 348 ms, sys: 9.94 ms, total: 358 ms
Wall time: 355 ms


In [29]:
%%time
def grader_5a(q5a):
    q5a_results  = pd.read_sql_query(q5a,conn)
    print(q5a_results.head(10))
    assert (q5a_results.shape == (4,2))

# query5a = """ *** Write your query for the question 5a *** """
query5a = """
WITH comb AS (
  SELECT *
  FROM m_cast mc 
  JOIN Person p ON p.PID = trim(mc.PID)
),
atleast_one_male_movies AS 
(
  SELECT 
      DISTINCT(MID), 
      Gender AS g, count(*) AS act_count 
  FROM comb
  GROUP BY trim(MID),Gender 
  HAVING Gender='Male' AND act_count >= 0
),
final_result_table AS 
(
  SELECT 
      m.year, 
      count(*) AS only_female_movie_count 
  FROM Movie m
  WHERE trim(m.MID) NOT IN (
                            SELECT
                                DISTINCT(MID) 
                            FROM atleast_one_male_movies
                            )
  GROUP BY m.year
  
)
SELECT * from final_result_table;
"""
grader_5a(query5a)

     year  only_female_movie_count
0    1939                        1
1    1999                        1
2    2000                        1
3  I 2018                        1
CPU times: user 144 ms, sys: 9.07 ms, total: 153 ms
Wall time: 152 ms


## Q5.b --- Now include a small change: report for each year the percentage of movies in that year with only female actors, and the total number of movies made that year. For example, one answer will be: 1990 31.81 13522 meaning that in 1990 there were 13,522 movies, and 31.81% had only female actors. You do not need to round your answer.

In [31]:
%%time
def grader_5b(q5b):
    q5b_results  = pd.read_sql_query(q5b,conn)
    print(q5b_results.head(10))
    assert (q5b_results.shape == (4,3))

# query5b = """ *** Write your query for the question 5b *** """
query5b = """
WITH comb AS (
  SELECT *
  FROM m_cast mc 
  JOIN Person p ON p.PID = trim(mc.PID)
),
atleast_one_male_movies AS 
(
  SELECT 
      DISTINCT(MID), 
      Gender AS g, count(*) AS act_count 
  FROM comb
  GROUP BY trim(MID),Gender 
  HAVING Gender='Male' AND act_count >= 0
),
final_result_table AS 
(
  SELECT 
      m.year, 
      count(*) AS only_female_movie_count 
  FROM Movie m
  WHERE trim(m.MID) NOT IN (
                            SELECT
                                DISTINCT(MID) 
                            FROM atleast_one_male_movies
                            )
  GROUP BY m.year
  
),
year_count AS
(
  SELECT 
      year, 
      count(*) AS m_count 
  FROM Movie m
  GROUP BY CAST(SUBSTR(TRIM(year),-4) AS INTEGER)
)

SELECT 
    frt.year AS YEAR , 
    CAST(frt.only_female_movie_count as REAL)/CAST(yc.m_count as REAL) as Percentage_Female_Only_Movie, 
    yc.m_count AS Total_Movies 
FROM final_result_table frt, year_count yc
WHERE CAST(SUBSTR(TRIM(frt.year),-4) AS INTEGER) = CAST(SUBSTR(TRIM(yc.year),-4) AS INTEGER)
;
"""
grader_5b(query5b)

     YEAR  Percentage_Female_Only_Movie  Total_Movies
0    1939                      0.500000             2
1    1999                      0.015152            66
2    2000                      0.015625            64
3  I 2018                      0.009615           104
CPU times: user 149 ms, sys: 5.97 ms, total: 155 ms
Wall time: 154 ms


## Q6 --- Find the film(s) with the largest cast. Return the movie title and the size of the cast. By "cast size" we mean the number of distinct actors that played in that movie: if an actor played multiple roles, or if it simply occurs multiple times in casts, we still count her/him only once.

In [32]:
%%time
def grader_6(q6):
    q6_results  = pd.read_sql_query(q6,conn)
    print(q6_results.head(10))
    assert (q6_results.shape == (3473, 2))

query6 = """ 
SELECT 
    m.title,count(*) AS count 
FROM M_Cast mc
INNER JOIN Movie m
ON trim(m.MID) = trim(mc.MID)
GROUP BY mc.MID
ORDER BY count(*) DESC;
"""
grader_6(query6)

                        title  count
0               Ocean's Eight    238
1                    Apaharan    233
2                        Gold    215
3             My Name Is Khan    213
4  Captain America: Civil War    191
5                    Geostorm    170
6                     Striker    165
7                        2012    154
8                      Pixels    144
9       Yamla Pagla Deewana 2    140
CPU times: user 46.8 s, sys: 23.1 ms, total: 46.8 s
Wall time: 46.7 s


## Q7 --- A decade is a sequence of 10 consecutive years. 
### For example, say in your database you have movie information starting from 1931. 
### the first decade is 1931, 1932, ..., 1940,
### the second decade is 1932, 1933, ..., 1941 and so on. 
### Find the decade D with the largest number of films and the total number of films in D.

In [33]:
%%time
def grader_7a(q7a):
    q7a_results  = pd.read_sql_query(q7a,conn)
    print(q7a_results.head(10))
    assert (q7a_results.shape == (78, 2))

# query7a = """ *** Write a query that computes number of movies in each year *** """
query7a = """
SELECT 
    year AS Movie_Year, 
    count(*) AS Total_Movies
FROM Movie
GROUP BY CAST(SUBSTR(TRIM(year),-4) AS INTEGER)
;
"""
grader_7a(query7a)

# using the above query, you can write the answer to the given question

  Movie_Year  Total_Movies
0       1931             1
1       1936             3
2       1939             2
3       1941             1
4       1943             1
5       1946             2
6       1947             2
7       1948             3
8       1949             3
9       1950             2
CPU times: user 9.64 ms, sys: 11 µs, total: 9.65 ms
Wall time: 8.88 ms


In [34]:
%%time
def grader_7b(q7b):
    q7b_results  = pd.read_sql_query(q7b,conn)
    print(q7b_results.head(10))
    assert (q7b_results.shape == (713, 4))

query7b = """   
WITH count_table AS
(
  SELECT 
      year AS Movie_Year, 
      count(*) AS Total_Movies
  FROM Movie
  GROUP BY CAST(SUBSTR(TRIM(year),-4) AS INTEGER)
)

SELECT 
    c1.Movie_Year as year1,
    c1.Total_Movies as total1,
    c2.Movie_Year as year2,
    c2.Total_Movies as total2
FROM count_table c1
JOIN count_table c2
ON CAST(SUBSTR(TRIM(c2.Movie_Year),-4) AS INTEGER) <= CAST(SUBSTR(TRIM(c1.Movie_Year),-4) AS INTEGER) + 9
AND CAST(SUBSTR(TRIM(c2.Movie_Year),-4) AS INTEGER) >= CAST(SUBSTR(TRIM(c1.Movie_Year),-4) AS INTEGER)
;
"""
grader_7b(query7b)
# if you see the below results the first movie year is less than 2nd movie year and 
# 2nd movie year is less or equal to the first movie year+9

# using the above query, you can write the answer to the given question

  year1  total1 year2  total2
0  1931       1  1931       1
1  1931       1  1936       3
2  1931       1  1939       2
3  1936       3  1936       3
4  1936       3  1939       2
5  1936       3  1941       1
6  1936       3  1943       1
7  1939       2  1939       2
8  1939       2  1941       1
9  1939       2  1943       1
CPU times: user 17.1 ms, sys: 0 ns, total: 17.1 ms
Wall time: 16.3 ms


In [35]:
%%time
def grader_7(q7):
    q7_results  = pd.read_sql_query(q7,conn)
    print(q7_results.head(10))
    assert (q7_results.shape == (1, 2))

query7 = """ *** Write a query that will return the decade that has maximum number of movies ***"""
query7 = """
WITH count_table AS
(
  SELECT 
      DISTINCT year AS Movie_Year, 
      count(*) AS Total_Movies
  FROM Movie
  GROUP BY CAST(SUBSTR(TRIM(year),-4) AS INTEGER)
),
joint AS
(
  SELECT 
    c1.Movie_Year as year1,
    c1.Total_Movies as total1,
    c2.Movie_Year as year2,
    c2.Total_Movies as total2
  FROM count_table c1
  JOIN count_table c2
  ON CAST(SUBSTR(TRIM(c2.Movie_Year),-4) AS INTEGER)  <=  CAST(SUBSTR(TRIM(c1.Movie_Year),-4) AS INTEGER) +9
  AND CAST(SUBSTR(TRIM(c2.Movie_Year),-4) AS INTEGER) > CAST(SUBSTR(TRIM(c1.Movie_Year),-4) AS INTEGER)
)

SELECT 
  year1 || '-' || CAST(CAST(SUBSTR(TRIM(year1),-4) AS INTEGER)+9 as TEXT) AS Decade, 
  CAST(total1 AS INTEGER) + SUM(total2) AS Decade_Movie_Count
FROM joint
GROUP BY year1
ORDER BY SUM(total2) DESC
LIMIT 1
;
"""
grader_7(query7)
# if you check the output we are printinng all the year in that decade, its fine you can print 2008 or 2008-2017

      Decade  Decade_Movie_Count
0  2008-2017                1203
CPU times: user 13.9 ms, sys: 0 ns, total: 13.9 ms
Wall time: 14.1 ms


## Q8 --- Find all the actors that made more movies with Yash Chopra than any other director.

In [36]:
%%time
def grader_8a(q8a):
    q8a_results  = pd.read_sql_query(q8a,conn)
    print(q8a_results.head(10))
    assert (q8a_results.shape == (73408, 3))

# Write a query that will results in number of movies actor-director worked together
query8a = """
 SELECT
     TRIM(MC.PID) ACTOR_PID,
     TRIM(MD.PID) DIRECTOR_PID,
     COUNT(DISTINCT TRIM(MD.MID)) AS NUM_OF_MOV
 FROM
     M_Cast MC,
     M_Director MD
 WHERE
     TRIM(MC.MID)= TRIM(MD.MID)
 GROUP BY
     ACTOR_PID,
     DIRECTOR_PID
;
"""
grader_8a(query8a)

# using the above query, you can write the answer to the given question

   ACTOR_PID DIRECTOR_PID  NUM_OF_MOV
0  nm0000002    nm0496746           1
1  nm0000027    nm0000180           1
2  nm0000039    nm0896533           1
3  nm0000042    nm0896533           1
4  nm0000047    nm0004292           1
5  nm0000073    nm0485943           1
6  nm0000076    nm0000229           1
7  nm0000092    nm0178997           1
8  nm0000093    nm0000269           1
9  nm0000096    nm0113819           1
CPU times: user 46.1 s, sys: 52.5 ms, total: 46.1 s
Wall time: 46 s


In [42]:
%%time

def grader_8(q8):
    q8_results  = pd.read_sql_query(q8,conn)
    print(q8_results.head(10))
    print(q8_results.shape)
    assert (q8_results.shape == (245, 2))

# query8 = """ *** Write a query that answers the 8th question ***"""
query8 = """
WITH temp AS
(
  SELECT 
      TRIM(mc.PID) AS Actor_id,
      TRIM(md.PID) AS Director_id 
  FROM
      M_Cast mc, 
      M_Director md 
  WHERE trim(mc.MID) = trim(md.MID)
),
joint_table AS 
(
  SELECT 
      Actor_id,Director_id, 
      count(*) AS m_count
  FROM temp
  GROUP BY Actor_id,Director_id
  ORDER BY m_count DESC
),
max_ad_movies AS
(
  SELECT 
      Actor_id AS Actor_id_max, 
      MAX(m_count) AS max_movie_count
  FROM joint_table
  GROUP BY Actor_id
),
most_worked_director AS
(
  SELECT 
      Actor_id,Director_id,m_count 
  FROM joint_table jt
  WHERE m_count = 
    (
      SELECT max_movie_count FROM max_ad_movies
      WHERE Actor_id_max = jt.Actor_id
    )
),
yash_chopra_pid AS
(
  SELECT trim(PID) AS p_id FROM Person WHERE trim(name) = 'Yash Chopra'
)

select
    p.name AS Name,
    mwd.m_count AS movie_count
FROM 
    most_worked_director mwd, 
    Person p
WHERE 
  trim(Director_id) IN (select p_id from yash_chopra_pid) AND
  trim(mwd.Actor_id) = trim(p.PID) 
;
"""
grader_8(query8)

                Name  movie_count
0        Jagdish Raj           11
1   Manmohan Krishna           10
2           Iftekhar            9
3      Shashi Kapoor            7
4      Rakhee Gulzar            5
5     Waheeda Rehman            5
6           Ravikant            4
7     Achala Sachdev            4
8        Neetu Singh            4
9      Leela Chitnis            3
(245, 2)
CPU times: user 1min 34s, sys: 109 ms, total: 1min 34s
Wall time: 1min 34s


## Q9 --- The Shahrukh number of an actor is the length of the shortest path between the actor and Shahrukh Khan in the "co-acting" graph. That is, Shahrukh Khan has Shahrukh number 0; all actors who acted in the same film as Shahrukh have Shahrukh number 1; all actors who acted in the same film as some actor with Shahrukh number 1 have Shahrukh number 2, etc. Return all actors whose Shahrukh number is 2.

In [43]:
%%time
def grader_9a(q9a):
    q9a_results  = pd.read_sql_query(q9a,conn)
    print(q9a_results.head(10))
    print(q9a_results.shape)
    assert (q9a_results.shape == (2382, 1))

query9a = """
WITH s1_movies AS
(
  select 
      DISTINCT trim(mc.MID) as MID
  from M_cast mc, Person p
  where 
      trim(mc.PID) = trim(p.PID) AND
      trim(p.name) LIKE 'Shah%rukh%khan'
),
srk_pid AS
(
  SELECT 
      trim(PID) as srk_id 
  FROM Person 
  WHERE trim(name) LIKE 'Shah%rukh%khan'
),
s1_actors AS 
(
  select 
      DISTINCT trim(mc.PID) as PID
  from M_Cast mc
  where 
    trim(mc.MID) IN (select srk_m.MID from s1_movies srk_m)
)
select * 
from s1_actors 
where PID not in (select srk_id from srk_pid)
;
"""
grader_9a(query9a)
# using the above query, you can write the answer to the given question

# selecting actors who acted with srk (S1)
# selecting all movies where S1 actors acted, this forms S2 movies list
# selecting all actors who acted in S2 movies, this gives us S2 actors along with S1 actors
# removing S1 actors from the combined list of S1 & S2 actors, so that we get only S2 actors

         PID
0  nm0004418
1  nm1995953
2  nm2778261
3  nm0631373
4  nm0241935
5  nm0792116
6  nm1300111
7  nm0196375
8  nm1464837
9  nm2868019
(2382, 1)
CPU times: user 61.3 ms, sys: 4.02 ms, total: 65.3 ms
Wall time: 64.5 ms


In [44]:
%%time
def grader_9(q9):
    q9_results  = pd.read_sql_query(q9,conn)
    print(q9_results.head(10))
    print(q9_results.shape)
    assert (q9_results.shape == (25698, 1))

query9 = """ 
WITH s1_movies AS
(
  select 
      DISTINCT trim(mc.MID) as MID
  from M_cast mc, Person p
  where trim(mc.PID) = trim(p.PID)
  and trim(p.name) LIKE 'Shah%rukh%khan'
),
s1_actors AS 
(
  select 
      DISTINCT trim(mc.PID) as PID
  from M_Cast mc
  where 
    trim(mc.MID) IN (select srk_m.MID from s1_movies srk_m)
),
s2_movies AS
(
  select 
      distinct trim(mc.MID) as MID
  from M_Cast mc
  where 
      trim(mc.PID) IN (select act_list_2.PID from s1_actors act_list_2)
),
s2_actors AS
(
 select 
    distinct trim(mc.PID) as PID
 from M_Cast mc
 where trim(mc.MID) IN (select mov_2_list.MID from s2_movies mov_2_list)
)

select 
    p.name from s2_actors act
INNER JOIN Person p
ON trim(p.PID) = trim(act.PID)
where act.PID NOT IN (select act_list_1.PID from s1_actors act_list_1)
;
"""
grader_9(query9)

                    Name
0        Alicia Vikander
1           Dominic West
2         Walton Goggins
3              Daniel Wu
4   Kristin Scott Thomas
5           Derek Jacobi
6     Alexandre Willaume
7           Tamer Burjaq
8         Adrian Collins
9         Keenan Arrison
(25698, 1)
CPU times: user 2min 54s, sys: 101 ms, total: 2min 54s
Wall time: 2min 54s


In [45]:
!jupyter nbconvert --to html "/content/SQL_ASSIGNMENT.ipynb"

[NbConvertApp] Converting notebook /content/SQL_ASSIGNMENT.ipynb to html
[NbConvertApp] Writing 340317 bytes to /content/SQL_ASSIGNMENT.html
